In [8]:
%load_ext autoreload
%autoreload 2
%run -i "prerequisites.py"

Exception ignored in: <function tunics.__del__ at 0x00000266B5120A60>
Traceback (most recent call last):
  File "C:\Users\mb16805\University of Bristol\grp-Multi 3 Chip Project - Documents\Multi_3_Chips\pycode_molly\Experimental code\tunicsqontrol.py", line 62, in __del__
    self.serial_port.close()
AttributeError: 'NoneType' object has no attribute 'close'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import ZeroLevel_Qontrol as q0
import FirstLevel_Qontrol as q1
import SecondLevel_Qontrol as q2
from powermeter import Powermeter as PM
import heater_dict as hd
import heater_characterisation_funcs as hcf
from tunicsqontrol import tunics
from scipy import optimize

pi=np.pi

In [3]:
# powermeter1 = Powermeter('PM100USB', serial='P2003916', unit='dBm', wavelength=1550) 
unit='W'
wavelength=1550
#Charlie powermeters 18/11/20:
PMa = PM('PM100USB', serial='P2007347',unit=unit, wavelength = wavelength)
PMb = PM('PM100USB', serial='P2010371',unit=unit, wavelength = wavelength) 
PMc = PM('PM100USB', serial='P2003683',unit=unit, wavelength = wavelength) 
PMd = PM('PM100USB', serial='P2008115',unit=unit, wavelength = wavelength)
#Alice powermeters 18/11/20:
PMe = PM('PM100USB', serial='P2005653',unit=unit, wavelength = wavelength)
PMf = PM('PM100USB', serial='P2010372',unit=unit, wavelength = wavelength) 
PMg = PM('PM100USB', serial='P2008116',unit=unit, wavelength = wavelength) 
PMh = PM('PM100USB', serial='P2007346',unit=unit, wavelength = wavelength)

In [5]:
#laser initialisation and wavelengths
serial_port_name = 'COM13'
mytunics = tunics(serial_port_name = serial_port_name)

p_wavelength = 1549.3
i_wavelength = 1558.98
s_wavelength = 1539.77


Connected to Tunics laser on serial port COM13



In [ ]:
#h=q2.heater_class(hd.board_list,hd.heaters)

In [3]:
%run -i "prerequisites.py"

## Phase stabilisation functions

In [15]:
def SineSq(x,amp,freq,phi_0,y_off):
    return amp*np.sin(freq*x+phi_0)**2+y_off

def FringeFitWithPlotting(spacing):
    #phase=list(np.linspace(0,2*pi,10))
    phase=[]
    PMePower=[]
    PMfPower=[]
    x=np.linspace(0, 2*pi, 30)
    time_fringe_started=time.time()
    for i in range(0,10):
        phase.append(i*spacing)
        h.set_phase('Phb1', i*spacing)
        time.sleep(0.15)
        PMePower.append(PMa.measure()*1000000)#times 1000000 bc curvefit doesn't work well with small numbers (micro watts)
        #PMfPower.append(PMf.measure()*1000000)
        #print('hello')
    #time with and without starting parameters, does it make it faster? what about limits?
    time_fringe_finished=int(time.time())
    initial_guess=[max(PMePower), 0.5, 0, min(PMePower)]
    param_bounds=([0, 0.3, -np.inf, 0], [2*max(PMePower), 0.8, np.inf, max(PMePower)]) # upper bound of amplitude could be given as related to the maximum power when not interfering?
    params, pcov = optimize.curve_fit(SineSq, phase, PMePower, bounds = param_bounds, p0 = initial_guess)
    plt.scatter(phase, PMePower)
    #plt.scatter(phase, PMfPower)
    plt.plot(x, SineSq(x, params[0], params[1], params[2], params[3]))
    plt.show()
    time_for_fringe=time_fringe_finished-time_fringe_started
    np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/modes14_{time_fringe_finished}_fringe.csv', np.vstack((phase, PMePower)).T,delimiter=',', header='phase,powers uW,time since epoch')
    np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/modes14_{time_fringe_finished}_fringe_full_fit_params.csv', list(params)+[time_for_fringe, spacing],delimiter=',', header='amp  freq  phi_0  y_off  time  spacing')
    print('parmams:', params)
    print('guess:', initial_guess)
    print('bounds:', param_bounds)
    return phase, PMePower, params, time_fringe_finished


def FringeFit(spacing):
    phase=[]
    PMePower=[]
    x=np.linspace(0, 2*pi, 30)
    time_fringe_started=time.time()
    for i in range(0,10):
        phase.append(i*spacing)
        h.set_phase('Phb1', i*spacing)
        time.sleep(0.15)
        PMePower.append(PMa.measure()*1000000)#times 1000000 bc curvefit doesn't work well with small numbers (micro watts)
    #time with and without starting parameters, does it make it faster? what about limits?
    time_fringe_finished=int(time.time())
    initial_guess=[max(PMePower), 0.5, 0, min(PMePower)]
    #print(max(PMePower))
    param_bounds=([0, 0.3, -np.inf, 0], [2*max(PMePower), 0.8, np.inf, max(PMePower)]) # upper bound of amplitude could be given as related to the maximum power when not interfering?
    params, pcov = optimize.curve_fit(SineSq, phase, PMePower, bounds = param_bounds, p0 = initial_guess)
    #time_for_fringe=time_fringe_finished-time_fringe_started
    #np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/modes14_{time_fringe_finished}_fringe.csv', np.vstack((phase, PMePower)).T,delimiter=',', header='phase,powers uW,time since epoch')
    #np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/modes14_{time_fringe_finished}_fringe_full_fit_params.csv', list(params),delimiter=',', header='amp  freq  phi_0  y_off')
    return params

In [6]:
#fringe fit to test multiple N and spacing values
def FF_testing_N():
    spacing_list=[pi/3,pi/3.3, pi/3.5, pi/3.8, pi/4, pi/4.5, pi/5, pi/6, pi/7, pi/8, pi/10]
    counter=0
    for spacing in spacing_list:
        params_output_array=[0,0,0,0]
        h.set_many_phases({'BS1':pi/2+0.25, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi+0.23, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi, 'MZIb2':0, 'MZIb3':pi/2, 'MZIb4':pi, 'Phb1':0})
        time.sleep(0.2)
        print(spacing)
        phase, PMePower, params, time_fringe_finished = FringeFitWithPlotting(spacing)
        initial_guess=[max(PMePower), 0.5, 0, min(PMePower)]
        param_bounds=([0, 0.3, -np.inf, 0], [2*max(PMePower), 0.8, np.inf, max(PMePower)])
        titles=['3point0', '3point3', '3point5', '3point8', '4point0', '4point5', '5point0', '6point0', '7point0', '8point0', '10point0']
        folder_titles=['pi by 3', 'pi by 3 point 3', 'pi by 3 point 5', 'pi by 3 point 8', 'pi by 4', 'pi by 4 point 5', 'pi by 5', 'pi by 6', 'pi by 7', 'pi by 8', 'pi by 10']
        for i in range(3,11):
            phase_N = phase[0:i]
            PMePower_N = PMePower[0:i]
            params, pcov = optimize.curve_fit(SineSq, phase_N, PMePower_N, bounds = param_bounds, p0 = initial_guess)
            params_output_array=np.column_stack([params_output_array,params])
        np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/{folder_titles[counter]}/modes14_{time_fringe_finished}_fringe_fit_params{titles[counter]}.csv', params_output_array,delimiter=',', header=' amp  freq  phi_0  y_off  spacing')
        counter+=1

## Modes 1+4

In [ ]:
mytunics.laser_switch('on')
mytunics.laser_wavelength(s_wavelength)
mytunics.laser_power(0)

In [ ]:
h.set_many_phases({'BS1':pi/2+0.25, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi+0.23, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi, 'MZIb2':0, 'MZIb3':pi/2, 'MZIb4':pi, 'Phb1':0})
time.sleep(0.2)

In [ ]:
for i in range(0,10):
    FF_testing_N()

In [ ]:
phase, PMePower, PMfPower, paramsE = FringeFitWithPlotting()
print(PMePower)
print(PMfPower)

In [ ]:
list1=[0,1,2,3]
list2=[4,5,6,7]
list3=np.column_stack([list1,list2])
list4=[8,9,10,11]
list3=np.column_stack([list3,list4])
print(list3)

In [ ]:
np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/PS Fringes/list_testing.csv', list3,delimiter=',', header=' amp  freq  phi_0  y_off  spacing')

## Phase drift with phase stabilisation

In [28]:
h.set_many_phases({'BS1':0, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi/2+0.23, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi/2, 'MZIb2':0, 'MZIb3':pi, 'MZIb4':pi})
#h.set_many_phases({'BS1':pi/2+0.25, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi+0.23, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi, 'MZIb2':0, 'MZIb3':3*pi/2, 'MZIb4':pi})
phase=0
withPS_time_array, withoutPS_time_array,PMe_array,PMf_array,noPS_arrayPMe,noPS_arrayPMf=[],[],[],[],[],[]
for i in range(0, 100):
    h.set_phase('Phb1', 0)
    time.sleep(0.1)
    withoutPS_time_array.append(time.time())
    noPS_arrayPMe.append(PMa.measure())
    noPS_arrayPMf.append(PMc.measure())
    params=FringeFit(pi/5) #zero_phase_power = first point of the 5 power points taken i.e. when Phb1=0
    h.set_phase('Phb1', -params[2]/params[1])
    time.sleep(0.1)#maybe try to make this longer? might be that the heater isn't quite set yet
    withPS_time_array.append(time.time())
    PMe_array.append(PMa.measure())
    PMf_array.append(PMc.measure())
    #time.sleep(0.5)

    
#saving_array = np.vstack((time_array, PMe_array, PMg_array)).T
saving_array = np.vstack((withoutPS_time_array, noPS_arrayPMe, noPS_arrayPMf, withPS_time_array, PMe_array, PMf_array)).T
np.savetxt(f'C:/Users/mb16805/University of Bristol/grp-Multi 3 Chip Project - Documents/Multi_3_Chips/pycode_molly/Experimental code/Experimental data/Molly testing phase stab 181120/with_and_without_PS_drift_over_time_{int(time.time())}piby5_10points.csv', saving_array,delimiter=',', header='#time,without PS output_1, without PS output_2, time, with PS output_1, with PS output_2, 0.1 sleep pi/5 separation 10 points')


0.00466356953
0.00458346783
0.0046376538200000005
0.00462351846
0.0046341197499999995
0.00456933202
0.00458111149
0.0045928914
0.00456108618
0.00427484004
0.0045127897
0.00455401805
0.00444328929
0.00445271287
0.0045622639
0.00439263692
0.00446095871
0.00423361168
0.00433491687
0.00453752635
0.00456933202
Caught error during powermeter read '<class 'pyvisa.errors.VisaIOError'>'. Tries remaining: 6.
Caught error during powermeter read '<class 'pyvisa.errors.VisaIOError'>'. Tries remaining: 5.
Caught error during powermeter read '<class 'pyvisa.errors.VisaIOError'>'. Tries remaining: 4.
Caught error during powermeter read '<class 'pyvisa.errors.VisaIOError'>'. Tries remaining: 3.
Caught error during powermeter read '<class 'pyvisa.errors.VisaIOError'>'. Tries remaining: 2.
Trying to reinitialise powermeter.
0.0102497433
0.0103215649
0.0102911306
0.0102205275
0.0102972182
0.010281392600000001
0.010216875299999999
0.010440859700000001
0.010448164099999999
0.0107719691
0.010509029199999999


RuntimeError: Error:Powermeter:measure: Tried 5 times to read powermeter, failed. Tried to reinitialise powermeter, failed.

In [17]:
mytunics.laser_switch('on')
mytunics.laser_wavelength(p_wavelength)
mytunics.laser_power(0)

In [16]:
h.set_many_phases({"BS1":pi,
    "BS2":pi,
    "BS3":0,
    "WDM1":0,
    "WDM2":pi/2,
    "WDM3":pi/2,
    "WDM4":pi/2,
    "Pre_RY1":pi,
    "Pre_RY2":pi,
    "Bell1":pi,
    "Bell2":pi,
    "Prj_RY1":pi,
    "Prj_RY2":pi/2,
    "Prj_RY3":pi,
    "Prj_RY4":pi/2,
    "Ctr_1":pi,
    "Ctr_2":pi,
    "Prj_RZ1":0,
    "Prj_RZ2":0,
    "Prj_RZ3":0,
    "Prj_RZ4":0,
    "Pre_RZ1":0,
    "Pre_RZ2":0,})

h.set_many_phases({"Atta1":0,
    "Atta2":0,
    "Atta3":pi,
    "Atta4":pi, #calibrated wrong way round
    "MZIa1":pi,
    "MZIa2":pi/2,
    "MZIa3":pi,
    "MZIa4":pi})

In [18]:
#h.set_many_phases({'BS1':pi/2+0.25, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi+0.23, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi, 'MZIb2':0, 'MZIb3':3*pi/2, 'MZIb4':pi})
phase=0
withPS_time_array, withoutPS_time_array,PMe_array,PMf_array,noPS_arrayPMe,noPS_arrayPMf=[],[],[],[],[],[]
for i in range(0, 100):
    h.set_phase('Phb3', 0)
    time.sleep(0.1)
    withoutPS_time_array.append(time.time())
    noPS_arrayPMe.append(PMf.measure())
    noPS_arrayPMf.append(PMh.measure())
    params=FringeFit(pi/5) #zero_phase_power = first point of the 5 power points taken i.e. when Phb1=0
    h.set_phase('Phb3', -params[2]/params[1])
    time.sleep(0.1)
    withPS_time_array.append(time.time())
    PMe_array.append(PMa.measure())
    PMf_array.append(PMc.measure())
    #time.sleep(0.5)

    
#saving_array = np.vstack((time_array, PMe_array, PMg_array)).T
saving_array = np.vstack((withoutPS_time_array, noPS_arrayPMe, noPS_arrayPMf, withPS_time_array, PMe_array, PMf_array)).T
np.savetxt(f'C:/Users/mb16805/University of Bristol/grp-Multi 3 Chip Project - Documents/Multi_3_Chips/pycode_molly/Experimental code/Experimental data/Molly testing phase stab 250221/with_and_without_PS_drift_over_time_{int(time.time())}piby5_10points.csv', saving_array,delimiter=',', header='#time,without PS output_1, without PS output_2, time, with PS output_1, with PS output_2, 0.1 sleep pi/5 separation 10 points')


In [23]:
h.set_many_phases({'BS1':0, 'BS2':0, 'BS3':0, 'WDM1':pi, 'WDM2':0, 'WDM4':pi, 'WDM3':0, 'Prj_RY1':pi/2, 'Prj_RY4':pi-0.02, 'Ctr_2':pi, 'Attb1':pi/2-0.25, 'Attb4':pi, 'MZIb1':pi/2, 'MZIb2':0, 'MZIb3':pi, 'MZIb4':pi})

In [ ]:
saving_array = np.vstack((time_array, PMe_array, PMg_array)).T
np.savetxt(f'C:/Users/IT072488-admin/University of Bristol/grp-Multi 3 Chip Project - Multi_3_Chips/pycode_molly/Experimental code/Experimental data/Interference/without_PS_drift_over_time_{int(time.time())}.csv', saving_array,delimiter=',', header='#time,output_1,output_2')